In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
import time
import copy

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

# Example Implementation

Here we provide a minimal working example demonstrating how to use the code to estimate the amplitude using the ESPIRIT algorithm.

In [6]:
# For reproducibility
np.random.seed(7)
# Set the per oracle noise parameter (See Eq. 18)
eta=1e-4
# Set the array parameters (See Thm. II.2 and Eq. 12) 
narray = [3, 3, 3, 3, 2, 2, 2, 2]
# Set the actual amplitude
a=0.1
theta = np.arcsin(a)

# This sets up the simulation that simulates the measured amplitudes at the various physical locations.
# It uses a C=1.5 value, which corresponds to the sampling schedule given in Eq. 16. The variable C here 
# is the parameter K in the paper.
ula_signal = TwoqULASignal(M=narray, C=1.5)
# Number of Monte Carlo trials used to estimate statistics. We tend to use 500 in the paper. Choose 100 here for speed.
num_mc = 100
thetas = np.zeros(num_mc, dtype = float)
errors = np.zeros(num_mc, dtype = float)

# Sets up the ESPIRIT object to estimate the amplitude
espirit = ESPIRIT()

for k in range(num_mc):
    # This estimates the covariance matrix of Eq. 8 using the approch given in DOI:10.1109/LSP.2015.2409153
    R = ula_signal.get_cov_matrix_toeplitz(theta, n_samples=ula_signal.n_samples, eta=eta)
    # This estimates the angle using the ESPIRIT algorithm
    theta_est = espirit.estimate_theta_toeplitz(R)
    # Estimate the error between estimated a and actual a
    error = np.abs(np.sin(theta)-np.sin(theta_est)) 
    thetas[k] = theta_est            
    errors[k] = error

# Compute the total number of queries
num_queries = 2*np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
# Compute the maximum single query
max_single_query = np.max(ula_signal.depths)

print(f'Array parameters: {narray}')
print(f'Number of queries: {num_queries}')
print(f'theta: {theta}')
print(f'Ave theta estimated: {np.mean(thetas)}')
print(f'a = {a}; a_est = {np.sin(np.mean(thetas))}')
print(f'Max Single Query: {max_single_query}')
print(f'99% percentile: {np.percentile(errors, 99):e}')
print(f'95% percentile: {np.percentile(errors, 95):e}')
print(f'68% percentile: {np.percentile(errors, 68):e}')
print()


Array parameters: [3, 3, 3, 3, 2, 2, 2, 2]
Number of queries: 9472
theta: 0.1001674211615598
Ave theta estimated: 0.10022172950892463
a = 0.1; a_est = 0.10005403597586164
Max Single Query: 648
99% percentile: 1.683732e-03
95% percentile: 9.665143e-04
68% percentile: 3.569322e-04

